In [3]:
import tensorflow as tf
import numpy as np
import time

In [4]:
@tf.function
def soft_thr(x,alpha):
    n = x.shape[0]
    S = tf.math.maximum(tf.abs(x)-alpha,0)*tf.math.sign(x)
    return S

In [5]:
#tf.debugging.set_log_device_placement(True)

In [7]:
def FISTA(beta, X, y, lamb, L, eta, tol = 1e-08, max_iter = 5000, dtype = tf.float32):
    n = X.shape[0]
    p = X.shape[1]
    crit = np.zeros(max_iter)
  
    with tf.device("/GPU:0"):
        dbeta = tf.constant(beta)
        dX = tf.constant(X)
        dy = tf.constant(y)
        t = tf.constant(1.0, dtype=dtype)
        dbeta_p = tf.constant(beta)
        dbeta_prev = tf.constant(beta)
        L_prev = tf.constant(L,dtype=dtype)

        for k in range(max_iter):
            dymXbp = dy-tf.linalg.matvec(dX,dbeta_p)
            drbp = tf.tensordot(dymXbp,dymXbp,axes=1)
            dXTrbp = tf.linalg.matvec(dX,dymXbp, 
                                    transpose_a=True)

            i_k = -1
            cond = True
            while cond:
                i_k += 1
                L_cur = L_prev*(eta**i_k)
                dbstar = dbeta_p + dXTrbp/L_cur
                dbeta = soft_thr(dbstar, lamb/L_cur)
                diff_beta = dbeta - dbeta_p
                RHS_1st = tf.tensordot(diff_beta, 
                                       diff_beta,axes=1)
                RHS_2nd = tf.tensordot(diff_beta, dXTrbp,
                                       axes=1)
                RHS = L_cur * RHS_1st.numpy() - 2.0 * RHS_2nd.numpy()
                dymXb = dy - tf.linalg.matvec(dX,dbeta)
                LHS = (tf.tensordot(dymXb,dymXb,
                       axes=1)).numpy()-drbp.numpy()
                cond = (LHS>RHS)
            L_prev = L_cur
            tnext =  (1.0+tf.sqrt(1+4*t**2))/2.0
            diff_beta = dbeta-dbeta_prev
            t1 = (t - 1.0)/tnext
            dbeta_p = dbeta+t1*diff_beta
            crit[k] = tf.norm(diff_beta) 
            if crit[k] < tol:
                break
            t = tnext
            dbeta_prev = dbeta
    return dbeta.numpy(), crit, k

## Double precision test

In [10]:
np.random.seed(2022)
n = 100; p = 200
xmat = np.random.normal(size = (n, p)).astype("float64")
beta = np.zeros(p)
beta[:int(0.05*p)] = 1.0
y = np.dot(xmat, beta) + np.random.normal(size = n)
beta_sol = np.zeros(p, dtype = "float64")

lamb = np.float64(np.sqrt(2 * np.log(p) / n) )
L = np.float64(10)
eta = np.float64(2)
tol = np.float64(1e-04)

In [11]:
t1 = time.time()
res = FISTA(beta_sol, xmat, y, lamb, L, eta, tol = tol, max_iter = 5000, dtype=tf.float64)
t2 = time.time()
print("Elapsed time: {:4f}".format(t2-t1))

Elapsed time: 4.311423


In [28]:
res[0].dtype

dtype('float64')

In [18]:
res[0][:20]

array([ 0.64070707,  1.11958681,  0.82775298,  1.11552658,  0.69531896,
        0.82641614,  0.73400349,  1.05100404,  0.91740151,  1.10698033,
        0.        , -0.26072158, -0.        , -0.        ,  0.        ,
        0.1230212 , -0.16857445, -0.        , -0.09391906,  0.        ])

## Single precision test

In [19]:
np.random.seed(2022)
n = 100; p = 200
xmat = np.random.normal(size = (n, p)).astype("float32")
beta = np.zeros(p, dtype = "float32")
beta[:int(0.05*p)] = 1.0
y = np.dot(xmat, beta) + np.random.normal(size = n).astype("float32")
beta_sol = np.zeros(p, dtype = "float32")


lamb = np.float32(np.sqrt(2 * np.log(p) / n) )
L = np.float32(10)
eta = np.float32(2)
tol = np.float32(1e-08)

In [20]:
beta = np.zeros(p, dtype = np.float32)
lam = np.float32(np.sqrt(2 * np.log(p) / n) )
L = np.float32(10)
eta = np.float32(2)
tol = np.float32(1e-08)
FISTA(beta, X, y, lam, L, eta, tol = 1e-08, max_iter = 5000)

Executing op Transpose in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ExpandDims in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Squeeze in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ExpandDims in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sque

2022-03-30 20:09:11.122381: I tensorflow/core/common_runtime/placer.cc:114] x: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2022-03-30 20:09:11.122407: I tensorflow/core/common_runtime/placer.cc:114] alpha: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2022-03-30 20:09:11.122416: I tensorflow/core/common_runtime/placer.cc:114] Abs: (Abs): /job:localhost/replica:0/task:0/device:GPU:0
2022-03-30 20:09:11.122423: I tensorflow/core/common_runtime/placer.cc:114] sub: (Sub): /job:localhost/replica:0/task:0/device:GPU:0
2022-03-30 20:09:11.122430: I tensorflow/core/common_runtime/placer.cc:114] Maximum: (Maximum): /job:localhost/replica:0/task:0/device:GPU:0
2022-03-30 20:09:11.122436: I tensorflow/core/common_runtime/placer.cc:114] Sign: (Sign): /job:localhost/replica:0/task:0/device:GPU:0
2022-03-30 20:09:11.122442: I tensorflow/core/common_runtime/placer.cc:114] mul: (Mul): /job:localhost/replica:0/task:0/device:GPU:0
2022-03-30 20:09:11.122449: I tensorflow/core/common_runt

Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_soft_thresh_27830 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Ex

Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ExpandDims in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Squeeze in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in 

Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Squeeze in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Less in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_soft_thresh_27830 in device /job:localhost/replica:0/task:0/device:GPU:0
Execu

...

Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ExpandDims in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Squeeze in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Less in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /

Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Less in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_soft_thresh_27830 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Exec

Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ExpandDims in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Squeeze in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in 

Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ExpandDims in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Squeeze in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Less in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in devi

Executing op Less in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Pow in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sqrt in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhos

In [22]:
res[0][:15]

array([ 0.3368621 ,  0.6961563 ,  0.39558616,  0.60575116,  0.5487673 ,
        0.49197343,  0.42253026,  0.7122659 ,  0.44804013,  0.54039085,
       -0.0174352 , -0.35006174, -0.13772216,  0.14593807, -0.02288086],
      dtype=float32)

In [23]:
res[2]

15

In [2]:
import tensorflow as tf
import numpy as np
import time

@tf.function
def soft_thr(x,alpha):
    n = x.shape[0]
    S = tf.math.maximum(tf.abs(x)-alpha,0)*tf.math.sign(x)
    return S

def FISTA(beta, X, y, lamb, L, eta, tol = 1e-08, max_iter = 5000, dtype = tf.float32):
    n = X.shape[0]
    p = X.shape[1]
    crit = np.zeros(max_iter)
  
    with tf.device("/GPU:0"):
        dbeta = tf.constant(beta)
        dX = tf.constant(X)
        dy = tf.constant(y)
        t = tf.constant(1.0, dtype=dtype)
        dbeta_p = tf.constant(beta)
        dbeta_prev = tf.constant(beta)
        L_prev = tf.constant(L,dtype=dtype)

        for k in range(max_iter):
            dymXbp = dy-tf.linalg.matvec(dX,dbeta_p)
            drbp = tf.tensordot(dymXbp,dymXbp,axes=1)
            dXTrbp = tf.linalg.matvec(dX,dymXbp, 
                                    transpose_a=True)

            i_k = -1
            cond = True
            while cond:
                i_k += 1
                L_cur = L_prev*(eta**i_k)
                dbstar = dbeta_p + dXTrbp/L_cur
                dbeta = soft_thr(dbstar, lamb/L_cur)
                diff_beta = dbeta - dbeta_p
                RHS_1st = tf.tensordot(diff_beta, 
                                       diff_beta,axes=1)
                RHS_2nd = tf.tensordot(diff_beta, dXTrbp,
                                       axes=1)
                RHS = L_cur * RHS_1st.numpy() - 2.0 * RHS_2nd.numpy()
                dymXb = dy - tf.linalg.matvec(dX,dbeta)
                LHS = (tf.tensordot(dymXb,dymXb,
                       axes=1)).numpy()-drbp.numpy()
                cond = (LHS>RHS)
            L_prev = L_cur
            tnext =  (1.0+tf.sqrt(1+4*t**2))/2.0
            diff_beta = dbeta-dbeta_prev
            t1 = (t - 1.0)/tnext
            dbeta_p = dbeta+t1*diff_beta
            crit[k] = tf.norm(diff_beta) 
            if crit[k] < tol:
                break
            t = tnext
            dbeta_prev = dbeta
    return dbeta.numpy(), crit, k


n = 100
p = 200

np.random.seed(2022)
X = np.random.randn(n,p).astype(np.float32)
tr_beta = np.zeros(p).astype(np.float32)
tr_beta[:int(0.05*p)] = 1.0
y = np.dot(X, tr_beta) + np.random.randn(n).astype(np.float32)

niter = 1
comp_time_tf = np.zeros(niter)

for i in np.arange(niter):

    beta = np.zeros(p, dtype = np.float32)
    lam = np.sqrt(2*np.log(p)/n).astype(np.float32)
    L = np.float32(10)
    eta = np.float32(2)
    
    t1 = time.time()
    out = FISTA(beta, X, y, lam, L, eta, tol = 1e-04, max_iter = 5000)
    t2 = time.time()
    comp_time_tf[i] = t2-t1
    
    print('\n ',i+1,'-th iteration, Collapsed Time: ', comp_time_tf[i])
   




  1 -th iteration, Collapsed Time:  5.9507060050964355
